# scratch

In [1]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr, _read_czi_slice
import zarr
from cloudvolume import CloudVolume
import json
from pathlib import Path
import aicspylibczi
import numpy as np

create zgroup

In [2]:
dir = Path(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/test-write-ome"
)
zgr_path = dir / "group-test"

zgr = zarr.group(zgr_path)

create array

In [12]:
sz = 64

for lvl in range(5):
    xysz = sz / 2**lvl
    zgr.zeros(
        str(lvl),
        shape=(sz, xysz, xysz),
        chunks=(4, np.amin((4, xysz)), np.amin((4, xysz))),
        dtype="<u2",
        dimension_separator="/",
    )

create zzattrs

In [13]:
axes = []
for dim in ["z", "x", "y"]:
    axes.append({"name": dim, "type": "space", "unit": "micrometer"})

datasets = []
for lvl in range(5):
    datasets.append(
        {
            "path": str(lvl),
            "coordinateTransformations": [
                {"type": "scale", "scale": [1.0, 2.0**lvl, 2.0**lvl]}
            ],
        }
    )

json_data = {
    "multiscales": [{"axes": axes, "datasets": datasets, "name": "/", "version": "0.4"}]
}

with open(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/test-write-ome/group-test/.zattrs",
    "w",
) as f:
    json.dump(json_data, f, indent=4)

## Read ome-zarr

In [ ]:
import ome_zarr
import ome_zarr.reader
import numpy as np

In [ ]:
file_path = "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/test_ngff.zarr"
loc = ome_zarr.io.parse_url(file_path, mode="r")
zarr_reader = ome_zarr.reader.Reader(loc).zarr
res0 = zarr_reader.load("0")

In [ ]:
full_image_npy = res0.compute()
print(full_image_npy.shape)

In [ ]:
sub_image = res0[:, :, :]
print(sub_image)
sub_image_npy = sub_image.compute()
print(np.unique(sub_image_npy))

## Explain vb

In [ ]:
from brainlit.algorithms.connect_fragments import explain_viterbrain
import pickle
import zarr

vb_path = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi/viterbrain.pickle"

with open(vb_path, "rb") as handle:
    vb = pickle.load(handle)

explain_viterbrain(vb, [29, 494, 59], [32, 375, 809])

In [ ]:
vb.nxGraph.nodes[194]